* Selectionner parmi les tables les colonnes qui de près ou de loin pourraient être intéressantes pour l'étude (d02_intermediate)

* Rechercher s'il existe des valeurs manquantes, les traiter (d02_intermediate)

* Rechercher s'il y a des colonnes qu'il faudrait transformer en datetime (d02_intermediate)

* Vérifier la consistence des données (d02_intermediate)

* Rechercher les valeurs abbérantes et les traiter (d02_intermediate)


# Nettoyage des données

## 0. Set up

### Mise en place de l'environnement

In [1]:
#Importation des Modules nécessaires
import pandas as pd
import sqlalchemy
import mysql.connector

#Pour que python reconnaisse un dossier comme un module il faut ajouter en son sein un fichier __init__.py 
#et également l'ajouter au PATH

import sys
sys.path.insert(0, "/home/apprenant/simplon_projects/american_dream/")


#On peut ensuite importer le dossier comme un module

from src.d00_utils.mysql_utils import mysql_connect, save_to_mysql
connect = mysql_connect()

### Importation des données depuis MySQL

In [3]:


df1 = pd.read_sql("excel_table",con=connect)
dfk = pd.read_sql("csv_table",con=connect)



## 1. Sélection des colonnes



J'affiche un échantillon de mes données pour analyser la pertinence des différentes colonnes


In [4]:


print(df1.head())
print(df1.columns)



   index  Survey Year           Timestamp  SalaryUSD        Country  \
0      0         2020 2020-01-04 18:50:34   115000.0  United States   
1      1         2020 2020-01-04 10:43:02   100000.0  United States   
2      2         2020 2020-01-04 09:51:46   100000.0          Spain   
3      3         2020 2020-01-04 01:08:54    70000.0  United States   
4      4         2020 2020-01-03 15:28:54   110000.0  United States   

  PostalCode       PrimaryDatabase  YearsWithThisDatabase  \
0         03  Microsoft SQL Server                     15   
1       None                 Other                      6   
2      28046  Microsoft SQL Server                      2   
3      94133  Microsoft SQL Server                      3   
4      95354                Oracle                     30   

                                      OtherDatabases    EmploymentStatus  ...  \
0  Microsoft SQL Server, MongoDB, Azure SQL DB (a...  Full time employee  ...   
1                                      MySQL


Je choisis uniquement les colonnes qui pourraient m'être utiles pour mon étude


In [9]:
df1 = df1[['index','SalaryUSD','Country','PostalCode','EmploymentStatus','JobTitle','ManageStaff','YearsWithThisTypeOfJob',
          'HowManyCompanies','OtherPeopleOnYourTeam', 'CompanyEmployeesOverall',
          'PopulationOfLargestCityWithin20Miles','EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear',
          'Gender', 'OtherJobDuties']]

Je récupère la liste des pays, pour vérifier qu'il n'y a pas de doublons.

In [27]:
countries = df1['Country'].unique()
countries.sort()
countries


array(['Albania', 'Anguilla', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Bahrain', 'Belarus', 'Belgium', 'Bermuda', 'Bolivia',
       'Brazil', 'Bulgaria', 'Canada', 'Cayman Islands', 'China',
       'Colombia', 'Costa Rica', 'Croatia', 'Czech Republic', 'Denmark',
       'Dominican Republic', 'Ecuador', 'El Salvador', 'Estonia',
       'Finland', 'France', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Guatemala', 'Guernsey', 'Hong Kong', 'Hungary', 'Iceland',
       'India', 'Indonesia', 'Iran', 'Ireland', 'Israel', 'Italy',
       'Jamaica', 'Jersey', 'Jordan', 'Kenya', 'Latvia', 'Lithuania',
       'Luxembourg', 'Macedonia', 'Malaysia', 'Maldives', 'Malta',
       'Mexico', 'Moldova', 'Nepal', 'Netherlands', 'New Zealand',
       'Nicaragua', 'Norway', 'Pakistan', 'Paraguay', 'Peru',
       'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar',
       'Romania', 'Russia', 'Saudi Arabia', 'Serbia and Montenegro',
       'Singapore', 'Slovakia', 'Slovenia', 'Sout

Il n'y a pas de doublons dans la liste des pays, je peux donc récupérer uniquement les lignes où la colonne Country est égale à 'United States' puisque c'est ce qui m'intéresse.

In [29]:
df1 = df1.loc[df1['Country'] == "United States"]
df1.head()

,index,SalaryUSD,Country,PostalCode,EmploymentStatus,JobTitle,ManageStaff,YearsWithThisTypeOfJob,HowManyCompanies,OtherPeopleOnYourTeam,CompanyEmployeesOverall,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties
0,0,115000.0,United States,03,Full time employee,DBA (Production Focus - build & troubleshoot s...,No,5,1 (this is the only company where I've had thi...,2,Not Asked,20K-99K (large town),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked
1,1,100000.0,United States,None,Full time employee,"Developer: App code (C#, JS, etc)",No,6,1 (this is the only company where I've had thi...,More than 5,Not Asked,1M+ (metropolis),Private business,Not Asked,Prefer not to say,Male,Not Asked
3,3,70000.0,United States,94133,Full time employee,Analyst,No,3,1 (this is the only company where I've had thi...,2,Not Asked,1M+ (metropolis),Private business,Not Asked,Change both employers and roles,Male,Not Asked
4,4,110000.0,United States,95354,Full time employee,DBA (General - splits time evenly between writ...,No,25,3,None,Not Asked,100K-299K (city),Non-profit,Not Asked,"Stay with the same employer, same role",Male,Not Asked
5,5,79500.0,United States,78727,Full time employee,Analyst,No,2,1 (this is the only company where I've had thi...,None,Not Asked,1M+ (metropolis),Private business,Not Asked,"Stay with the same employer, same role",Male,Not Asked




Je fais la même chose pour la seconde table


In [22]:
print(dfk.head())
print(dfk.columns)

   index  Unnamed: 0                                          Job Title  \
0      0           0  Data Analyst, Center on Immigration and Justic...   
1      1           1                               Quality Data Analyst   
2      2           2  Senior Data Analyst, Insights & Analytics Team...   
3      3           3                                       Data Analyst   
4      4           4                             Reporting Data Analyst   

              Salary Estimate  \
0  $37K-$66K (Glassdoor est.)   
1  $37K-$66K (Glassdoor est.)   
2  $37K-$66K (Glassdoor est.)   
3  $37K-$66K (Glassdoor est.)   
4  $37K-$66K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Are you eager to roll up your sleeves and harn...     3.2   
1  Overview\n\nProvides analytical and technical ...     3.8   
2  We’re looking for a Senior Data Analyst who ha...     3.4   
3  Requisition NumberRR-0001939\nRemote:Yes\nWe c...     4.1   
4  ABOUT FANDUEL GROUP\n\nFanD

In [31]:
dfk = dfk[['Job Title', 'Salary Estimate',
       'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters',
       'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue'
]]


## 2. Valeurs manquantes
# 
Recherche des valeurs manquantes


In [32]:

print(df1.isnull().sum()) 
print(df1.shape)

index                                     0
SalaryUSD                                 0
Country                                   0
PostalCode                              466
EmploymentStatus                          0
JobTitle                                  0
ManageStaff                               0
YearsWithThisTypeOfJob                    0
HowManyCompanies                          0
OtherPeopleOnYourTeam                     0
CompanyEmployeesOverall                   0
PopulationOfLargestCityWithin20Miles      0
EmploymentSector                          0
LookingForAnotherJob                      0
CareerPlansThisYear                       0
Gender                                    0
OtherJobDuties                          385
dtype: int64
(5680, 17)




Il me manque 1390 veleurs pour PostalCode et 620 pour OtherJobDUties sur un total de 8627 lignes. Ces deux colonnes ne sont pas au centre de mon études donc ça se vaut pas le cout de supprimer toute la ligne pour elles

(Après sélection des valeurs pour USA uniquement: il manque 466 valeurs dans PostalCode (environ 8%) et 385 dans OtherJobDUties (moins de 7%))


### traitement de Postal code


Cela ne ferait pas de sens de remplacer le code postal par un code postal arbitraire


### Traitement de OtherJobDuties

In [13]:

print(df1.OtherJobDuties.unique())
print(df1.OtherJobDuties.value_counts())



, etc)'
 'Developer: App code (C#, JS, etc), Developer: Business Intelligence (SSRS, PowerBI, etc), Developer: T-SQL, Engineer, Manager'
 'Analyst, Architect, DBA (Development Focus - tunes queries, indexes, does deployments), DBA (Production Focus - build & troubleshoot servers, HA/DR), Developer: App code (C#, JS, etc), Developer: T-SQL, Manager'
 'Analyst, Architect, DBA (Development Focus - tunes queries, indexes, does deployments), DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers), Developer: App code (C#, JS, etc), Developer: T-SQL, Engineer, Manager'
 'Analyst, Architect, DBA (Development Focus - tunes queries, indexes, does deployments), Developer: Business Intelligence (SSRS, PowerBI, etc), Developer: T-SQL, Engineer, Manager'
 'Analyst, DBA (Development Focus - tunes queries, indexes, does deployments), Developer: App code (C#, JS, etc), Developer: T-SQL, Manager'
 'Data Scientist, DBA (General - splits time evenly betw



Comme il y a trop de valeur unique différente pour OtherJobDuties, je ne peux remplacer les valeurs manquantes par une valeur standard


### Conclusion sur les valeurs manquantes


Je choisis donc de conserver l'ensemble des données dans ma base mais je note un point de vigilance quand j'aurais à utiliser les variables codePostal et OtherJobDuties


### 2ème table

In [34]:
print(dfk.isnull().sum()) 
print(dfk.shape)

Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         1
Location             0
Headquarters         0
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
dtype: int64
(2253, 13)


## 3. Traitement des dates


In [16]:
print(df1.dtypes)

index                                     int64
SalaryUSD                               float64
Country                                  object
PostalCode                               object
EmploymentStatus                         object
JobTitle                                 object
ManageStaff                              object
YearsWithThisTypeOfJob                    int64
HowManyCompanies                         object
OtherPeopleOnYourTeam                    object
CompanyEmployeesOverall                  object
PopulationOfLargestCityWithin20Miles     object
EmploymentSector                         object
LookingForAnotherJob                     object
CareerPlansThisYear                      object
Gender                                   object
OtherJobDuties                           object
dtype: object




Aucune de mes colonnes ne correspond à une date, je n'ai donc pas à faire de traitement


## 4. Traitement des doublons

In [17]:
print(df1.duplicated().value_counts())

False    8627
dtype: int64




Aucune des colonnes de ma base n'est dupliquée, je vérifie cependant que c'est également le cas sur la colonne index


In [18]:
print(df1.duplicated('index').value_counts())

False    8627
dtype: int64




On est bon.


## to be continued

### Avant de partir je sauvegarde mon travail dans my sql


In [21]:
save_to_mysql(db_connect=connect,df_to_save=df1,df_name='excel_table_v1.0')